In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
import pandas as pd
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

In [12]:
word_to_vec_map = {}
word_to_index = {}
index_to_word = {}
with open('glove.6B.50d.txt', encoding = 'utf8' ) as f:
    for i, line in enumerate(f):
        element=line.split()
        word = element[0]
        vector_values = np.array(element[1:], dtype=float)
        word_to_index [word] = (i+1)
        index_to_word [i+1] = word
        word_to_vec_map [word] = vector_values
        


In [106]:
dataset = pd.read_csv('Restaurant_Reviews.tsv', delimiter = '\t', quoting=3 ) # 3 means we will remove " in reviews
X_train =dataset.iloc[:800, 0].values
y_train = dataset.iloc[:800,1].values
X_test =dataset.iloc[800:, 0].values
y_test = dataset.iloc[800:,1].values
y_train.shape

(800,)

In [107]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [108]:
y_train.shape

(800, 2)

In [96]:
y_train.shape

(800, 1)

In [45]:
sentence_len = []
for i in range(X_train.shape[0]):
    review = re.sub('[^a-zA-Z]', ' ', X_train[i])
    review = review.lower()
    X_train[i] = review
    words = [word for word in X_train[i].split()]
    sentence_len.append(len(words)) 
    
maxLen = max(sentence_len)

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
max_num_words = 40000
embedding_dim=len(word_to_vec_map['the'])

In [70]:
tokenizer = Tokenizer(num_words=max_num_words)
tokenizer.fit_on_texts(X_train)

In [71]:
train_sequences = tokenizer.texts_to_sequences(X_train)

In [72]:
train_sequences = pad_sequences(train_sequences, maxlen=maxLen, padding='post')

In [73]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences = pad_sequences(test_sequences, maxlen=maxLen, padding='post')

In [52]:
word_index = tokenizer.word_index

In [63]:
word_index['this']

9

In [59]:
all_embs = np.stack(word_to_vec_map.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

C:\Users\masou\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


(0.020940489508694315, 0.6441042976813115)

In [64]:
num_words = len(word_index) +1
embedding_matrix = np.random.normal(emb_mean, emb_std, (num_words, embedding_dim))
for word, i in word_index.items():
    if i >= max_num_words:
        break
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be random.
        embedding_matrix[i] = embedding_vector

In [65]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional
from keras.layers.embeddings import Embedding

In [102]:
model = Sequential()
model.add(Embedding(num_words, embedding_dim, trainable=True, weights=[embedding_matrix]))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128, return_sequences=True, recurrent_dropout=0.1, dropout=0.1), 'concat'))
model.add(Dropout(0.3))
model.add(LSTM(128, return_sequences=False, recurrent_dropout=0.1, dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2, activation='softmax'))

In [103]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 50)          101150    
_________________________________________________________________
dropout_5 (Dropout)          (None, None, 50)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 256)         183296    
_________________________________________________________________
dropout_6 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
__________

In [104]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [109]:
model.fit(train_sequences, y_train, epochs = 16,
          batch_size = 64, shuffle=True,
          validation_data=[test_sequences, y_test])

Instructions for updating:
Use tf.cast instead.
Train on 800 samples, validate on 200 samples
Epoch 1/16
800/800 [==============================] - 6s 8ms/step - loss: 0.7003 - acc: 0.5138 - val_loss: 0.8323 - val_acc: 0.2400
Epoch 2/16
800/800 [==============================] - 2s 2ms/step - loss: 0.6925 - acc: 0.5500 - val_loss: 0.7826 - val_acc: 0.2450
Epoch 3/16
800/800 [==============================] - 2s 2ms/step - loss: 0.6912 - acc: 0.5487 - val_loss: 0.6815 - val_acc: 0.7600
Epoch 4/16
800/800 [==============================] - 1s 2ms/step - loss: 0.6927 - acc: 0.5575 - val_loss: 0.7199 - val_acc: 0.2550
Epoch 5/16
800/800 [==============================] - 2s 2ms/step - loss: 0.6911 - acc: 0.5737 - val_loss: 0.7512 - val_acc: 0.2600
Epoch 6/16
800/800 [==============================] - 2s 2ms/step - loss: 0.6845 - acc: 0.5625 - val_loss: 0.7790 - val_acc: 0.2800
Epoch 7/16
800/800 [==============================] - 2s 2ms/step - loss: 0.6900 - acc: 0.5687 - val_loss: 0.7584 

In [110]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc'])

In [112]:
model.fit(train_sequences, y_train, epochs = 16,
          batch_size = 64, shuffle=True,
          validation_data=[test_sequences, y_test])

Train on 800 samples, validate on 200 samples
Epoch 1/16
800/800 [==============================] - 6s 7ms/step - loss: 0.5578 - acc: 0.7413 - val_loss: 0.7026 - val_acc: 0.6250
Epoch 2/16
800/800 [==============================] - 1s 2ms/step - loss: 0.5226 - acc: 0.7550 - val_loss: 0.7143 - val_acc: 0.6300
Epoch 3/16
800/800 [==============================] - 1s 2ms/step - loss: 0.5201 - acc: 0.7550 - val_loss: 0.7712 - val_acc: 0.5800
Epoch 4/16
800/800 [==============================] - 1s 2ms/step - loss: 0.5005 - acc: 0.7675 - val_loss: 0.6325 - val_acc: 0.7400
Epoch 5/16
800/800 [==============================] - 1s 2ms/step - loss: 0.5076 - acc: 0.7400 - val_loss: 0.7309 - val_acc: 0.6150
Epoch 6/16
800/800 [==============================] - 1s 2ms/step - loss: 0.5330 - acc: 0.7450 - val_loss: 0.9017 - val_acc: 0.5150
Epoch 7/16
800/800 [==============================] - 2s 2ms/step - loss: 0.5160 - acc: 0.7400 - val_loss: 0.7234 - val_acc: 0.6350
Epoch 8/16
800/800 [==========

In [5]:
X_train[9].split()

['a', 'great', 'touch']

In [6]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words =[word.lower() for word in X[i].split()]
        
        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            if w in word_to_index.keys():
                

                # Set the (i,j)th entry of X_indices to the index of the correct word.
                X_indices[i, j] = word_to_index[w]
                # Increment j to j + 1
                j = j+1
            
    
    return X_indices

In [21]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it non-trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False )
    


    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [92]:
# GRADED FUNCTION: Emojify_V2

def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(input_shape, dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(1)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [93]:
model = Emojify_V2((maxlen,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 40)                0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 40, 50)            20000050  
_________________________________________________________________
lstm_9 (LSTM)                (None, 40, 128)           91648     
_________________________________________________________________
dropout_9 (Dropout)          (None, 40, 128)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
__________

In [96]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [89]:
maxlen=40

In [90]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxlen)

In [86]:
X_train_indices

array([[1.4398e+04, 4.1470e+03, 3.8000e+01, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.4795e+04, 1.5000e+01, 3.7000e+01, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.7000e+01, 2.5110e+04, 6.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [1.2510e+03, 4.2000e+01, 1.6000e+01, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.0000e+00, 1.1160e+03, 1.2220e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.2800e+02, 2.0000e+01, 8.4000e+01, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00]])

In [97]:
model.fit(X_train_indices, y, epochs = 50, batch_size = 32, shuffle=True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50


InvalidArgumentError: Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 0 1 0 1 0 1 0 1 0 1 0 0 0 1 1 0 0 1 0 1 1 1 0 0 1 0 0 1 0 0 1 0
	 [[{{node loss_5/activation_5_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}}]]

In [73]:
X_train_indices

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [46]:
sentences_to_indices(np.array(['I love you']), word_to_index, 10)

array([[ 42., 836.,  82.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])

In [29]:
sentence_to_avg('I love you', word_to_vec_map)

array([-0.00701397,  0.54196333, -0.19225433, -0.52465667,  0.77827667,
       -0.04985667, -0.31802333,  0.1746592 , -0.52433   ,  0.47641919,
       -0.33454333,  0.93489   , -0.61863667, -0.164186  ,  1.10000667,
        0.33991333,  0.29203   ,  0.35769333,  0.07931167, -0.724163  ,
       -0.42256667,  0.87212   ,  0.70861667,  0.45412333,  1.2277    ,
       -2.0613    , -1.31806667,  0.23561667,  1.2105    , -1.26068   ,
        3.33396667,  0.74604667, -0.60947   ,  0.23688667, -0.31138667,
       -0.179042  ,  0.17087667,  0.119286  ,  0.35114667, -0.56632333,
        0.09226223, -0.03197267, -0.20612   ,  0.41710333,  0.168862  ,
        0.18619333,  0.08125467, -0.80106333, -0.20057967,  0.78087   ])

In [13]:
word=words[0]

In [14]:
word

'the'

In [21]:
vector=np.array(words[1:], dtype=float)

In [22]:
vector

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01])

In [2]:
dataset = pd.read_csv('Restaurant_Reviews.tsv', delimiter = '\t', quoting=3 ) # 3 means we will remove " in reviews
y = dataset.iloc[:,-1].values

In [3]:
dataset.isnull().sum()  # Checking the missing data

Review    0
Liked     0
dtype: int64

In [4]:
### Cleaning text - stemming - getting the root - group different version of same words- tokenization-- split the review to
###  to words--
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\masou\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
corpus = []
for i in range(dataset.shape[0]):
    review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
    review = review.lower()
    review = [ps.stem(word) for word in review.split() if  word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [6]:
# Creating the Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)    # To decrease the matrix sparsity, we decide to consider the 1500 most common words!
X = cv.fit_transform(corpus).toarray()

In [7]:
# Fitting the Naive Bayes to training data
from sklearn.model_selection import train_test_split
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, )

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)


GaussianNB(priors=None, var_smoothing=1e-09)

In [8]:
#Predicting the test results
y_pred = classifier.predict(X_test)

In [9]:
# Creating the confusion matrix to chech the accuracy, precision, and F1 scores
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [10]:
cm

array([[55, 42],
       [12, 91]], dtype=int64)

In [2]:
import GloVe

ModuleNotFoundError: No module named 'GloVe'